In [1]:
from music21 import *
import glob
import pickle
import numpy as np
from keras.utils import np_utils

### Data Preparation

In [2]:
def generatenotes():
    """ Extract songs from midi file and save all notes into string format """
    notes = []

    for file in glob.glob("midi_songs2/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        
        if parts: 
            notes_to_parse = parts.parts[0].recurse()
        else: 
            notes_to_parse = midi.flat.notes
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes

In [3]:
def sequence(notes,n_vocab):
    """ Create input and output sequences """
    
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    sequence_length = 50
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
       
    n_patterns = len(network_input)     
    
    # Reshape the input into a format compatible with LSTM layers and normalize
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    n_vocab = len(set(notes))
    network_input = network_input / n_vocab
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input,network_output)

### Train Model

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

In [5]:
def create_network(network_input, n_vocab):
    """ Create network structure """
    model = Sequential()

    model.add(LSTM(
    500,
    return_sequences=True,
    time_major=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(500, return_sequences=True))
    model.add(Dropout(0.3)) 
    model.add(LSTM(500))
    model.add(Dense(250))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation("softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])

    return model

In [6]:
def train(model, network_input, network_output):
    """ Train the model """
    
    filepath = "weights.{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor="loss",
        verbose=0,
        save_best_only=True,
        mode="min"
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=40, batch_size=64, callbacks=callbacks_list)

In [7]:
def train_network():
    
    """ This function calls all other functions and trains the LSTM"""
    
    notes = generatenotes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = sequence(notes, n_vocab)

    model = create_network(network_input,n_vocab)

    train(model, network_input, network_output)

In [8]:
train_network()

Epoch 1/40
33/33 [==============================] - 74s 2s/step - loss: 4.6395
Epoch 2/40
33/33 [==============================] - 75s 2s/step - loss: 4.2705
Epoch 3/40
33/33 [==============================] - 71s 2s/step - loss: 4.2419
Epoch 4/40
33/33 [==============================] - 71s 2s/step - loss: 4.2237
Epoch 5/40
33/33 [==============================] - 71s 2s/step - loss: 4.2004
Epoch 6/40
33/33 [==============================] - 70s 2s/step - loss: 4.2007
Epoch 7/40
33/33 [==============================] - 70s 2s/step - loss: 4.1934
Epoch 8/40
33/33 [==============================] - 69s 2s/step - loss: 4.1941
Epoch 9/40
33/33 [==============================] - 70s 2s/step - loss: 4.1779
Epoch 10/40
33/33 [==============================] - 69s 2s/step - loss: 4.1773
Epoch 11/40
33/33 [==============================] - 69s 2s/step - loss: 4.1755
Epoch 12/40
33/33 [==============================] - 69s 2s/step - loss: 4.1736
Epoch 13/40
33/33 [==============================